In [ ]:
import os

In [ ]:
from keras.preprocessing import image

In [ ]:
import cv2

In [ ]:
categories = ['with_mask' , 'without_mask']

In [ ]:
base_path = 'machine_l\\fmd\data\\train'

In [ ]:
data = []
for category in categories:
  path = os.path.join(base_path , category)
  label = categories.index(category)

  for file in os.listdir(path):
    img_path  = os.path.join(path , file)
    img = cv2.imread(img_path)
    img = cv2.resize(img , (224,224))

    data.append([img, label])

In [ ]:
print(len(data))

In [ ]:
import random

In [ ]:
random.shuffle(data)

In [ ]:
x = []
y = []

for i , j in data:
  x.append(i)
  y.append(j)

In [ ]:
import numpy as np

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x = x/255

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(x, y , test_size = 0.2)

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
vgg = VGG19(weights = 'imagenet',
            include_top = False,
            input_shape = (224,224,3))

In [ ]:
vgg.summary()

In [ ]:
from keras import Sequential

In [ ]:
model = Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
  model.add(layer) 

In [ ]:
for i in model.layers:
  i.trainable = False

In [ ]:
from keras.layers import Dense
from keras.layers import Flatten

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(256, activation='relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss = 'binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs = 10 , validation_data = (x_test, y_test))

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
def detect_face_mask(img):
    # y_pred = np.argmax(model.predict(img.reshape(1,224,224,3)),axis=1)  
    y_pred = model.predict(img.reshape(1,224,224,3))
   
 
    return y_pred

In [ ]:
while True:
  ret, frame = cap.read()
  # call the detection method
  img = cv2.resize(frame, (224,224))
  y_pred = detect_face_mask(img)

  coods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

  for x , y , w, h in coods:
    cv2.rectangle(frame,(x,y),(x+w , y+h), (255,0,0) , 3)

  if y_pred[0][0] == float(1):
  
    draw_label(frame,"no mask" , (30, 30), (0,0,255))
  else:
    
    draw_label(frame, "mask" , (30,30) , (0,255,0))

  cv2.imshow("Frame" , frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
cv2.destroyAllWindows()

In [ ]:
haar = cv2.CascadeClassifier('machine_l\\fmd\\haarcascade_frontalface_alt.xml')

In [ ]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [ ]:
def draw_label(img, text , pos , bg_color):
    text_size = cv2.getTextSize(text ,cv2.FONT_HERSHEY_SIMPLEX , 1 , cv2.FILLED)
    end_x = pos[0] + text_size[0][0] 
    end_y = pos[1] + text_size[0][1] 
    cv2.rectangle(img,pos,(end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)